In [1]:
from fastai.tabular.all import *
import pandas as pd
import numpy as np
import pickle
import os

In [2]:
symbols, dfs = [], []
training_path = 'E:/BitBot/training_data/'
for filename in os.listdir(training_path):
    symbols.append(filename.replace('.csv', ''))
    dfs.append(pd.read_csv(training_path + filename))

In [3]:
for idx, symbol in enumerate(symbols):
    print(symbol, dfs[idx].shape)

BCHUSDT (45462, 79)
BNBUSDT (42955, 79)
BTCUSDT (24393, 79)
BTTUSDT (50652, 79)
CHZUSDT (73922, 79)
DOGEUSDT (62568, 79)
EOSUSDT (44122, 79)
ETCUSDT (52015, 79)
ETHUSDT (36848, 79)
LINKUSDT (60245, 79)
LTCUSDT (45484, 79)
MATICUSDT (73541, 79)
NEOUSDT (51166, 79)
THETAUSDT (73361, 79)
TRXUSDT (42003, 79)
VETUSDT (67357, 79)
XLMUSDT (52465, 79)
XRPUSDT (51318, 79)


In [4]:
if False:
    splits = RandomSplitter(valid_pct=0.2)(range_of(df))
else:
    splits = [[], []]
    offset = 0
    for df in dfs:
        length = df.shape[0]
        split_pos = int(length * 0.9)
        splits[0].extend(list(range(offset, offset + split_pos)))
        splits[1].extend(list(range(offset + split_pos, offset + length)))
        offset += length
df = pd.concat(dfs)

In [5]:
cat_names = list(df.columns)[-len(symbols)-1:-1]
cont_names = list(df.columns)[:-len(symbols)-1]
y_names = list(df.columns)[-1:]

In [6]:
to = TabularPandas(df, procs=[Categorify], cat_names=cat_names, cont_names=cont_names, y_names=y_names, splits=splits)

In [7]:
dataloader = to.dataloaders(bs=2**14)

In [77]:
learn = tabular_learner(dataloader, metrics=rmse)

In [ ]:
learn.lr_find()

In [61]:
learn.fit_one_cycle(1, lr_max=6e-3)

epoch,train_loss,valid_loss,_rmse,time
0,1.016328,1.039118,1.019371,00:08


In [74]:
learn.fit_one_cycle(4, lr_max=6e-2)

epoch,train_loss,valid_loss,_rmse,time
0,1.049504,1.059081,1.029117,00:07
1,1.008033,1.020303,1.010100,00:07
2,0.994070,1.020679,1.010287,00:07
3,0.986788,1.018481,1.009198,00:07


In [78]:
learn.fit_one_cycle(10, lr_max=6e-2)

epoch,train_loss,valid_loss,_rmse,time
0,1.024228,1.038421,1.019029,00:07
1,1.004320,1.015360,1.007651,00:07
2,0.993553,1.021373,1.010630,00:07
3,0.985010,1.029855,1.014818,00:07
4,0.976558,1.025905,1.012869,00:07
5,0.968069,1.056035,1.027635,00:07
6,0.957623,1.066301,1.032618,00:07
7,0.945283,1.071975,1.035362,00:07
8,0.932318,1.065628,1.032292,00:07
9,0.920976,1.073353,1.036028,00:07


In [ ]:
learn.show_results(max_n=50)

In [ ]:
learn.export('model_all_2021-07-06.pickle')

In [62]:
df_train, df_val = df.iloc[splits[0]], df.iloc[splits[1]]

In [63]:
dl_train = DataLoader(dataset=df.iloc[splits[0]])
df_val = DataLoader(dataset=df.iloc[splits[1]])

In [64]:
#to = TabularPandas(df, procs=[Categorify], cat_names=cat_names, cont_names=cont_names, y_names=y_names, splits=splits)
#dls = TabularDataLoaders.from_df(df, procs=[Categorify], cat_names=cat_names, cont_names=cont_names, y_names=y_names)

In [79]:
dl_train = learn.dls.test_dl(df_train)
dl_val = learn.dls.test_dl(df_val)

In [80]:
pred_train, gt_train = learn.get_preds(dl=dl_train)
pred_val, gt_val = learn.get_preds(dl=dl_val)
import pickle
with open('preds.pickle', 'wb') as f:
    pickle.dump({
        'pred_train': pred_train.squeeze(),
        'gt_train': gt_train.squeeze(),
        'pred_val': pred_val.squeeze(),
        'gt_val': gt_val.squeeze()
    }, f)

In [53]:
pred_val.squeeze().shape

torch.Size([94998])

In [ ]:
indices = (pred > 0.2).nonzero()[:, 0]
pred_a, gt_a = pred[indices], gt[indices]

In [ ]:
from matplotlib import pyplot as plt
plt.plot(pred_a)
plt.plot(gt_a)